## Project Setup

In [ ]:
# all import statements needed for the project

import os
import re
import bs4
import matplotlib.pyplot as plt
import pandas as pd
import requests
import pyarrow.parquet as pq
import geopandas as gpd
import warnings
from typing import List, Tuple, Optional
import sqlite3
import numpy as np
import folium
from folium.plugins import HeatMap

In [ ]:
TLC_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

In [ ]:
#make sure the QUERY_DIRECTORY exists
try:
    os.mkdir(QUERY_DIRECTORY)
except Exception as e:
    if e.errno == 17:
        #the directory already exists
        pass
    else:
        raise

## Part 1: Data Preprocessing

### Common Functions

In [ ]:
def get_html() -> bytes:
    """
    Request the HTML content of TLC_URL webpage.

    Returns:
        bytes: The HTML content of the webpage in bytes type.
    """
    response = requests.get(TLC_URL)
    html = response.content
    
    return html

In [ ]:
def find_taxi_parquet_links()-> List[str]:
    """
    Finds and returns a list of URLs of Yellow Taxi parquet files.

    Returns:
        List[str]: a list of strings representing URLs of Yellow Taxi 
        parquet files.
    """
    parquet_links = list()
    
    html = get_html()
    soup = bs4.BeautifulSoup(html, "html.parser")
    links = soup.find_all("a")
    #using re module to help me extract links for Yellow Taxi 
    pattern = re.compile(r"Yellow Taxi Trip Records")
    for link in links:
        title = link.get('title')
        if title != None:
            match = pattern.search(title)
            if match:
                #collect URLs of Yellow Taxi parquet files
                parquet_links.append(link.get('href'))
                
    return parquet_links

In [ ]:
def find_HVFHV_parquet_links() -> List[str]:
    """
    Finds and returns a list of URLs of High Volume FHV parquet files.

    Returns:
        List[str]: a list of strings representing URLs of High Volume FHV 
            parquet files.
    """
    parquet_links = list()
    
    html = get_html()
    soup = bs4.BeautifulSoup(html, "html.parser")
    links = soup.find_all("a")
    #using re module to extract links for High Volume For-Hire Vehicle 
    pattern = re.compile(r"High Volume For-Hire Vehicle Trip Records")
    for link in links:
        title = link.get('title')
        if title != None:
            match = pattern.search(title)
            if match:
                #collect URLs of HVFHV parquet files
                parquet_links.append(link.get('href'))
                
    return parquet_links

### Downloading Taxi files

In [ ]:
def download_taxi_parquet_files():
    """
    Downloads Yellow Taxi parquet files from URLs found by 'find_taxi_
    parquet_links()'.

    Returns:
        None
    """
    taxi_files = find_taxi_parquet_links()
    for file_url in taxi_files:
        file_url = file_url.replace(' ', '')
        name = file_url.split('trip-data/')[1]
        #check if the file already exists
        if os.path.exists(name):
            pass
        else:
            response = requests.get(file_url, stream=True)
            with open(name, "wb") as f:
                for chunk in response.iter_content(chunk_size=1024): 
                    if chunk:
                        f.write(chunk)

In [ ]:
#call function to download yellow_taxi_files programmatically
#download_taxi_parquet_files()

### Downloading HVFHV files

In [ ]:
def download_HVFHV_parquet_files() -> None:
    """
    Downloads High Volume FHV parquet files from URLs found by 'find_
    HVFHV_parquet_links'.

    Returns:
        None
    """
    taxi_files = find_HVFHV_parquet_links()
    for file_url in taxi_files:
        file_url = file_url.replace(' ', '')
        name = file_url.split('trip-data/')[1]
        #check if the file already exists
        if os.path.exists(name):
            pass
        else:
            response = requests.get(file_url, stream=True)
            with open(name, "wb") as f:
                for chunk in response.iter_content(chunk_size=1024): 
                    if chunk:
                        f.write(chunk)  

In [ ]:
#call function to download HVFHV_parquet_files programmatically
#download_HVFHV_parquet_files()

### Load Taxi Zones

In [ ]:
def load_taxi_zones(shapefile: str) -> gpd.GeoDataFrame:
    """
    Read a shapefile containing taxi zones into a GeoDataFrame.

    Parameters:
        shapefile(str):
            The path to the shapefile that contains the taxi zones data.

    Returns:
        gpd.GeoDataFrame: A GeoDataFrame that contains the taxi zones data.
    """
    data = gpd.read_file(shapefile)
    
    return data

In [ ]:
#load and store taxi zones data
loaded_taxi_zones = load_taxi_zones('taxi_zones.shp')
loaded_taxi_zones

In [ ]:
def lookup_coords_for_taxi_zone_id(zone_loc_id: int, loaded_taxi_zones: gpd.GeoDataFrame) -> Tuple[float, float]:
    """
    Lookup the coordinates of for a given taxi zone Location ID.

    Parameters:
        zone_loc_id(int): The location ID for the taxi zone
        loaded_taxi_zones(gpd.GeoDataFrame):A GeoDataFrame that contains taxi zone geometries.

    Returns:
        Tuple[float, float]: A tuple containing the longitude and latitude 
        of the given location ID. 
        Returns (None, None) if the ID is not found.
    """
    row = loaded_taxi_zones[loaded_taxi_zones['LocationID']==zone_loc_id]
    if (len(row)!=0):
        #coordinate reference system 4326
        row = row.to_crs(4326)
        #extract centroid
        centroid = row.geometry.centroid.iloc[0]

        return centroid.x, centroid.y
    else:
        
        return None, None

### Calculate Sample Size

In [ ]:
def calculate_sample_size(p: float, e: float, z: float) -> float:
    """
    Calculate the sample size using Cochran's Formula.

    Parameters:
        p (float): the (estimated) proportion of the population.
        e (float): the desired level of precision.
        z (float): Z values based on the confidence level.

    Returns:
        float: The sample size calculated by Cochran's Formula.
    """
    n0 = ((z**2)*p*(1-p))/(e**2)
    
    return n0

### Process Taxi Data

In [ ]:
def get_and_clean_taxi_month(url: str) -> pd.DataFrame:
    """
    Extract a sample of taxi data from the dataset of given url and clean it.

    Parameters:
        url (str): The URL of the Parquet file containing the Yellow Taxi data.

    Returns:
        pd.DataFrame: A cleaned DataFrame containing the relevant columns and data.
    """

    #get file's name from url
    file_url = url.replace(' ', '')
    name = file_url.split('trip-data/')[1]

    #only extract a specific sample size of data
    sample = int(calculate_sample_size(p=0.5, e=0.05, z=1.96))
    #read a sample of data by calling its name
    data = pq.read_table(name).to_pandas().sample(n=sample, random_state=1)
    
    #extract specific columns we want (there are three types of dataframes)

    #the first type of dataset
    if ('VendorID' in data.columns):
        #clean values in relevant columns before adding them up as a total surcharge
        data = data.rename(columns={'airport_fee': 'Airport_fee', 'Airport_Fee': 'Airport_fee'})
        data['Airport_fee'] = data['Airport_fee'].fillna(0)
        data = data.dropna(subset=['improvement_surcharge', 'congestion_surcharge', 'extra', 'tip_amount'], how='any')
        data = data[~(data[['improvement_surcharge', 'congestion_surcharge', 'extra', 'tip_amount', 'Airport_fee']]< 0).any(axis=1)]
        
        #define new columns for storing information(coordinates and surcharge)
        data['pickup_longitude'] = None
        data['pickup_latitude'] = None
        data['dropoff_longitude'] = None
        data['dropoff_latitude'] = None
        data['surcharge'] = None
        # use shp to find lat and lon for pickup/dropoff location
        drop_index = list()
        for index, row in data.iterrows():
            lon, lat = lookup_coords_for_taxi_zone_id(row['PULocationID'],loaded_taxi_zones)
            lon2, lat2 = lookup_coords_for_taxi_zone_id(row['DOLocationID'],loaded_taxi_zones)
            if (lon != None and lat != None and lon2 != None and lat2 != None):
                data.loc[index,'pickup_longitude'] = lon
                data.loc[index,'pickup_latitude'] = lat
                data.loc[index,'dropoff_longitude'] = lon2
                data.loc[index,'dropoff_latitude'] = lat2
            else:
                #store rows where pickup locationID or dropoff locationID is not valid
                drop_index.append(index)
            #calculate total surcharge of each taxi ride
            data.loc[index,'surcharge'] = row['improvement_surcharge'] + row['congestion_surcharge'] + row['extra'] + row['tip_amount'] + row['Airport_fee']
        #drop rows where pickup locationID or dropoff locationID is not valid    
        data = data.drop(drop_index)
        #only keep columns needed 
        data = data[['VendorID','tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_distance','pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude','fare_amount', 'surcharge', 'mta_tax', 'tolls_amount', 'tip_amount']]       

    #the second type of dataset
    if ('vendor_id' in data.columns):
        #clean values in relevant columns before adding them up as a total surcharge
        data = data.dropna(subset=['surcharge', 'tip_amount'], how='any')
        #calculate total surcharge of each taxi ride   
        data['surcharge2'] = data['surcharge'] + data['tip_amount']
        #only keep columns needed 
        data = data[['vendor_id','pickup_datetime','dropoff_datetime', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'fare_amount', 'surcharge2', 'mta_tax', 'tolls_amount', 'tip_amount']]

    #the third type of dataset
    if ('vendor_name' in data.columns):
        #clean values in relevant columns before adding them up as a total surcharge
        data = data.dropna(subset=['surcharge', 'Tip_Amt'], how='any')  
        #calculate total surcharge of each taxi ride    
        data['surcharge2'] = data['Tip_Amt'] + data['surcharge']
        #only keep columns needed 
        data = data[['vendor_name','Trip_Pickup_DateTime','Trip_Dropoff_DateTime', 'Trip_Distance', 'Start_Lon', 'Start_Lat', 'End_Lon', 'End_Lat', 'Fare_Amt', 'surcharge2', 'mta_tax', 'Tolls_Amt', 'Tip_Amt']]

    #normalize column names
    new_column_names = ['VendorID', 'pickup_datetime', 'dropoff_datetime', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'fare_amount', 'surcharge', 'taxes', 'tolls_amount', 'tip_amount']
    data.columns = new_column_names

    #normalize appropriate column types
    data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'])
    data['dropoff_datetime'] = pd.to_datetime(data['dropoff_datetime'])
    data['surcharge'] = pd.to_numeric(data['surcharge'])

    #removing trips that start and/or end outside (40.560445, -74.242330) and (40.908524, -73.717047)
    data = data[(data['pickup_latitude'] >= 40.560445) & (data['pickup_latitude'] <= 40.908524) &
                (data['pickup_longitude'] >= -74.242330) & (data['pickup_longitude'] <= -73.717047) &
                (data['dropoff_latitude'] >= 40.560445) & (data['dropoff_latitude'] <= 40.908524) &
                (data['dropoff_longitude'] >= -74.242330) & (data['dropoff_longitude'] <= -73.717047)]

    #removing a distance of 0
    drop_index2 = list()
    for index, row in data.iterrows():
        #drop rows with same pickup and dropoff location
        if ((row['pickup_longitude']==row['dropoff_longitude']) and (row['pickup_latitude']==row['dropoff_latitude'])):
            drop_index2.append(index)
    data = data.drop(drop_index2)
    #drop rows with distance of 0 
    data = data[data['trip_distance']>0]

    #removing invalid data points (NaN)
    data = data.dropna()

    return data

In [ ]:
def get_and_clean_taxi_data(parquet_urls: List[str])-> pd.DataFrame:
    """
    Load and clean data from a list of URLs for Yellow Taxi Parquet files, and 
    combine them into one large DataFrame.

    Parameters:
        parquet_urls (list): A list of URLs of Parquet files.

    Returns:
        pd.DataFrame: A DataFrame containing all cleaned taxi data.
    """
    all_taxi_dataframes = []
    
    #read and clean data from every month
    for parquet_url in parquet_urls:
        #only consider files for yellow taxi
        if ('yellow' in parquet_url):
            dataframe = get_and_clean_taxi_month(parquet_url)
            all_taxi_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.concat(all_taxi_dataframes)
    
    return taxi_data

In [ ]:
def get_taxi_data() -> pd.DataFrame:
    """
    Find the links for all taxi data Parquet files, cleans the data, and returns 
    the DataFrame containing all the taxi data.

    Returns:
        pd.DataFrame: A DataFrame containing all the cleaned yellow-taxi data.
    """

    all_taxi_urls = find_taxi_parquet_links()
    taxi_data = get_and_clean_taxi_data(all_taxi_urls)

    return taxi_data

In [ ]:
# The cleaned and complete taxi data
warnings.filterwarnings('ignore')
taxi_data = get_taxi_data()

In [ ]:
taxi_data.head()

In [ ]:
taxi_data.info()

In [ ]:
taxi_data.describe()

### Processing Uber Data

In [ ]:
def get_and_clean_uber_month(url: str) -> pd.DataFrame:
    """
    Processes, and cleans Uber trip data for a given month.It downloads Uber 
    trip data from the specified URL, filters the data, normalizes columns, 
    and removes invalid or out-of-bound data. 

    Parameters:
        url (str): The URL of the Uber trip data file.

    Returns:
        pd.DataFrame: A cleaned DataFrame containing filtered and normalized Uber trip data.
    """
    
    file_url = url.replace(' ', '')
    name = file_url.split('trip-data/')[1]
    data = pq.read_table(name).to_pandas()

    #filter for Uber license
    data = data[data['hvfhs_license_num'] == "HV0003"]

    #sample the data
    sample = int(calculate_sample_size(p=0.5, e=0.05, z=1.96))
    data = data.sample(n=sample, random_state=1)

    #add geographic coordinates for pickup and dropoff locations
    if 'hvfhs_license_num' in data.columns:
        coords = data.apply(
            lambda row: pd.Series(
                lookup_coords_for_taxi_zone_id(row['PULocationID'], loaded_taxi_zones) +
                lookup_coords_for_taxi_zone_id(row['DOLocationID'], loaded_taxi_zones)
            )
            if (row['PULocationID'] in loaded_taxi_zones['LocationID'] and
                row['DOLocationID'] in loaded_taxi_zones['LocationID'])
            else pd.Series([None, None, None, None]),
            axis=1
    )
    coords.columns = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']
    data = pd.concat([data, coords], axis=1)
    data = data.dropna(subset=['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'])

    #select columns
    data = data[[
             'request_datetime',
             'pickup_datetime',
             'dropoff_datetime',
             'trip_miles',
             'trip_time',
             'pickup_longitude',
             'pickup_latitude',
             'dropoff_longitude',
             'dropoff_latitude',
             'base_passenger_fare',
             'tolls',
             'bcf',
             'sales_tax',
             'congestion_surcharge',
             'airport_fee',
             'tips'
                ]
            ] 

    #normalize appropriate column types - time
    data['request_datetime'] = pd.to_datetime(data['request_datetime'])
    data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'])
    data['dropoff_datetime'] = pd.to_datetime(data['dropoff_datetime'])
    
    #normalize appropriate column types - number
    data['trip_miles'] = pd.to_numeric(data['trip_miles'])
    data['trip_time'] = pd.to_numeric(data['trip_time'])
    data['base_passenger_fare'] = pd.to_numeric(data['base_passenger_fare'])
    data['tolls'] = pd.to_numeric(data['tolls'])
    data['bcf'] = pd.to_numeric(data['bcf'])
    data['sales_tax'] = pd.to_numeric(data['sales_tax'])
    data['congestion_surcharge'] = pd.to_numeric(data['congestion_surcharge'])
    data['airport_fee'] = pd.to_numeric(data['airport_fee'])
    data['tips'] = pd.to_numeric(data['tips'])
    
    #clean relevant columns before adding them up as a total surcharge
    data['airport_fee'] = data['airport_fee'].fillna(0)
    data = data.dropna(subset=['airport_fee', 'congestion_surcharge', 'bcf', 'tips'], how='any')
    #compute total surcharge
    data['all_surcharge'] = data['congestion_surcharge'] + data['airport_fee'] + data['bcf'] + data['tips']
    
    #remove NaN
    data = data.dropna()
    
    #normalize column names (pu, do, trip_time, bcf)
    new_column_names = [
             'request_datetime',
             'pickup_datetime',
             'dropoff_datetime',
             'trip_miles',
             'trip_time_seconds',
             'pickup_longitude',
             'pickup_latitude',
             'dropoff_longitude',
             'dropoff_latitude',
             'base_passenger_fare',
             'tolls',
             'black_car_fund',
             'sales_tax',
             'congestion_surcharge',
             'airport_fee',
             'tips',
             'all_surcharge'
                       ]
    data.columns = new_column_names

    #remove invalid time
    data = data[data['pickup_datetime'] < data['dropoff_datetime']]
    #remove 0 distance
    data = data[data['trip_miles'] > 0]
    #remove 0 time
    data = data[data['trip_time_seconds'] > 0]
    
    #removing trips that start and/or end outside (40.560445, -74.242330) and (40.908524, -73.717047)
    data = data[(data['pickup_latitude'] >= 40.560445) & (data['pickup_latitude'] <= 40.908524) &
                (data['pickup_longitude'] >= -74.242330) & (data['pickup_longitude'] <= -73.717047) &
                (data['dropoff_latitude'] >= 40.560445) & (data['dropoff_latitude'] <= 40.908524) &
                (data['dropoff_longitude'] >= -74.242330) & (data['dropoff_longitude'] <= -73.717047)]

    return data

In [ ]:
def get_and_clean_uber_data(parquet_urls: List[str]) -> pd.DataFrame:
    """
    Load and clean data from a list of URLs for fhvhv files, and combine them 
    into one large DataFrame.

    Parameters:
        parquet_urls (List[str]): A list of URLs for Parquet files containing Uber data.

    Returns:
        pd.DataFrame: A cleaned and combined DataFrame containing Uber trip data 
        from all specified Parquet files.
    """
    
    all_uber_dataframes = []
    
    #process each URL in the list
    for parquet_url in parquet_urls:
        #only consider fhvhv data
        if ('fhvhv' in parquet_url):
            dataframe = get_and_clean_uber_month(parquet_url)
            all_uber_dataframes.append(dataframe)
        
    uber_data = pd.concat(all_uber_dataframes)
    
    return uber_data

In [ ]:
def get_uber_data() -> pd.DataFrame:
    """
    Retrieves all relevant uber URLs from the TLC page, and combined all cleaned datas as one DataFrame.

    Returns:
        pd.DataFrame: A cleaned DataFrame containing Uber trip data from all relevant Parquet files.
    """
    all_uber_urls = find_HVFHV_parquet_links()
    uber_data = get_and_clean_uber_data(all_uber_urls)
    return uber_data

In [ ]:
warnings.filterwarnings('ignore')
uber_data = get_uber_data()

In [ ]:
uber_data.head()

In [ ]:
uber_data.info()

In [ ]:
uber_data.describe()

### Processing Weather Data

In [ ]:
def get_all_weather_csvs() -> List[pd.DataFrame]:
    """
    Reads weather data for the years 2020 to 2024 and returns a list of 
    DataFrames for each year.

    Returns:
        List[pd.DataFrame]: A list of DataFrames containing weather data for each year.
    """
    all_weather_files = list()
    #iterate each year from 2020 to 2024
    for i in range(2020,2025,1):
        #read weather data from specific year
        weather_file = pd.read_csv(f'{i}'+'_weather.csv')
        all_weather_files.append(weather_file)
    
    return all_weather_files

In [ ]:
def clean_month_weather_data_hourly(csv_file: pd.DataFrame) -> pd.DataFrame:
    """
    Cleans weather data by extracting the hour from the 'DATE' column, fixing invalid values in 
    'HourlyPrecipitation' and 'HourlyWindSpeed', and return the cleaned data in DataFrame.

    Parameter:
        csv_file (pd.DataFrame): The input file of date containing weather information.

    Returns:
        pd.DataFrame: A cleaned DataFrame with 'DATE', 'hour', 'HourlyPrecipitation', and 'HourlyWindSpeed'.
    """

    hour_data = csv_file.copy()
    #convert date in datetime foramt and extract hour
    hour_data['DATE'] = pd.to_datetime(hour_data['DATE'])
    hour_data['hour'] = hour_data['DATE'].dt.hour
    hour_data['DATE'] = hour_data['DATE'].dt.strftime('%Y-%m-%d')
    hour_data['DATE'] = pd.to_datetime(hour_data['DATE'])
    
    #fix invalid values for HourlyPrecipitation
    hour_data['HourlyPrecipitation'] = hour_data['HourlyPrecipitation'].replace('T', 0)
    hour_data['HourlyPrecipitation'] = hour_data['HourlyPrecipitation'].str.replace('s', '')
    hour_data['HourlyPrecipitation'] = hour_data['HourlyPrecipitation'].fillna(0)
    hour_data['HourlyPrecipitation'] = pd.to_numeric(hour_data['HourlyPrecipitation'])
    #fix invalid values for HourlyWindSpeed
    hour_data['HourlyWindSpeed'] = hour_data['HourlyWindSpeed'].fillna(0)
    hour_data['HourlyWindSpeed'] = pd.to_numeric(hour_data['HourlyWindSpeed'])
    
    hour_data = hour_data[['DATE','hour','HourlyPrecipitation','HourlyWindSpeed']].reset_index(drop=True)
    
    return hour_data

In [ ]:
def clean_month_weather_data_daily(csv_file: pd.DataFrame) -> pd.DataFrame:
    """
    Cleans daily weather data by extracting daily record time, fixing invalid values, and
    returning a cleaned DataFrame with relevant columns.

    Parameters:
        csv_file (pd.DataFrame): The input DataFrame containing weather data.

    Returns:
        pd.DataFrame: A cleaned DataFrame with 'DATE', 'DailyPrecipitation', 
        'DailyAverageWindSpeed', and 'DailySnowfall'.
    """
    daily_data = csv_file.copy()
    #convert 'DATE' to datetime format and extract weather information when time is 23:59
    daily_data['DATE'] = pd.to_datetime(daily_data['DATE'])
    daily_data = daily_data[daily_data['DATE'].dt.strftime('%H:%M') == '23:59']
    daily_data['DATE'] = daily_data['DATE'].dt.strftime('%Y-%m-%d')
    daily_data = daily_data[~(daily_data.duplicated(['DATE'], keep=False) & (daily_data['DailyPrecipitation'].isna()) & (daily_data['DailyAverageWindSpeed'].isna()) & (daily_data['DailySnowfall'].isna()))]
    daily_data['DATE'] = pd.to_datetime(daily_data['DATE'])
    
    #fix invalid values for DailyPrecipitation
    daily_data['DailyPrecipitation'] = daily_data['DailyPrecipitation'].replace('T', 0)
    daily_data['DailyPrecipitation'] = daily_data['DailyPrecipitation'].fillna(0)
    daily_data['DailyPrecipitation'] = pd.to_numeric(daily_data['DailyPrecipitation'])

    #fix invalid values for DailyAverageWindSpeed
    daily_data['DailyAverageWindSpeed'] = daily_data['DailyAverageWindSpeed'].fillna(0)
    daily_data['DailyAverageWindSpeed'] = pd.to_numeric(daily_data['DailyAverageWindSpeed'])

    #fix invalid values for DailyAverageWindSpeed
    daily_data['DailySnowfall'] = daily_data['DailySnowfall'].replace('T', 0)
    daily_data['DailySnowfall'] = daily_data['DailySnowfall'].fillna(0)
    daily_data['DailySnowfall'] = pd.to_numeric(daily_data['DailySnowfall'])
    
    daily_data = daily_data[['DATE','DailyPrecipitation','DailyAverageWindSpeed','DailySnowfall']].reset_index(drop=True)
    
    return daily_data

In [ ]:
def load_and_clean_weather_data() -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Loads and cleans weather data into hourly and daily dataframes from CSV files.
    
    Returns:
        Tuple[pd.DataFrame, pd.DataFrame]: A tuple containing hourly_data and daily_data
    """

    weather_csv_files = get_all_weather_csvs()
    
    hourly_dataframes = []
    daily_dataframes = []
    
    #receives hourly weather data and daily weather data for each year
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes that combines all hourly or daily data 
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

In [ ]:
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

In [ ]:
hourly_weather_data.head()

In [ ]:
hourly_weather_data.info()

In [ ]:
hourly_weather_data.describe()

In [ ]:
daily_weather_data.head()

In [ ]:
daily_weather_data.info()

In [ ]:
daily_weather_data.describe()

## Part 2: Storing Cleaned Data

In [ ]:
connection = sqlite3.connect(":memory:")

In [ ]:
 #use SQLite3 to create 4 tables
HOURLY_WEATHER_SCHEMA = """
Create Table IF NOT EXISTS hourly_weather (
    id INTEGER PRIMARY KEY,
    DATE DATETIME,
    hour INTEGER,
    HourlyPrecipitation REAL,
    HourlyWindSpeed REAL
);
"""

DAILY_WEATHER_SCHEMA = """
Create Table IF NOT EXISTS daily_weather (
    id INTEGER PRIMARY KEY,
    DATE DATETIME,
    DailyPrecipitation REAL,
    DailyAverageWindSpeed REAL,
    DailySnowfall REAL
);
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi_trips (
    id INTEGER PRIMARY KEY,
    pickup_datetime DATETIME,
    dropoff_datetime DATETIME,
    trip_distance REAL,
    pickup_longitude REAL,
    pickup_latitude REAL,
    dropoff_longitude REAL,
    dropoff_latitude REAL,
    fare_amount REAL,
    surcharge REAL,
    taxes REAL,
    tolls_amount REAL,
    tip_amount REAL
);
"""

UBER_TRIPS_SCHEMA = """
Create Table IF NOT EXISTS uber_trips (
    id INTEGER PRIMARY KEY,
    request_datetime DATETIME,
    driver_arrived_datetime DATETIME,
    pickup_datetime DATETIME,
    dropoff_datetime DATETIME,
    trip_miles REAL,
    trip_time_seconds INTEGER,
    pickup_longitude REAL,
    pickup_latitude REAL,
    dropoff_longitude REAL,
    dropoff_latitude REAL,
    base_passenger_fare REAL,
    tolls REAL,
    black_car_fund REAL,
    sales_tax REAL,
    congestion_surcharge REAL,
    airport_fee REAL,
    tips REAL,
    all_surcharge REAL
);
"""

In [ ]:
#create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [ ]:
#create the tables with the schema files
with open(DATABASE_SCHEMA_FILE, 'r') as f:
    schema_sql = f.read()
    sql_statement = schema_sql.split(';')
        
#execute the SQL commands to create the tables
with connection:
    for statement in sql_statement:
        connection.execute(statement)

### Add Data to Database

In [ ]:
def write_dataframes_to_table(table_to_df_dict: Dict[str, pd.DataFrame], connection: sqlite3.Connection) -> None:
    """
    Inserts rows from the provided DataFrames into their corresponding database tables.

    Args:
        table_to_df_dict (Dict[str, pd.DataFrame]): 
            A dictionary where the keys are table names and the values are pandas DataFrames 
            containing data to insert into the respective tables.
        connection (sqlite3.Connection): 
            A connection object for the SQLite database.

    Returns:
        None

    Notes:
        This function handles "taxi_trips" table, "hourly_weather" table, 
        "daily_weather" table, and "uber_trips" table.
    """
    for table_name in table_to_df_dict:
        #add data to Taxi database
        if (table_name == 'taxi_trips'):
            df = table_to_df_dict[table_name]
            insert_query = """
            INSERT INTO taxi_trips (pickup_datetime, dropoff_datetime, 
                                    trip_distance,
                                    pickup_longitude, pickup_latitude, 
                                    dropoff_longitude, dropoff_latitude,
                                    fare_amount, surcharge, taxes, tolls_amount, tip_amount)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """
    
            #loop through DataFrame rows and insert each row into the table
            data = []
            for index, row in df.iterrows():
                pickup_datetime = row['pickup_datetime'].strftime('%Y-%m-%d %H:%M:%S') 
                dropoff_datetime = row['dropoff_datetime'].strftime('%Y-%m-%d %H:%M:%S') 
                data.append((
                pickup_datetime, dropoff_datetime,
                row['trip_distance'],
                row['pickup_longitude'], row['pickup_latitude'],
                row['dropoff_longitude'], row['dropoff_latitude'],
                row['fare_amount'], row['surcharge'], row['taxes'], row['tolls_amount'], row['tip_amount']
                ))
        
            with connection:
                connection.executemany(insert_query,data)

        #add data to hourly weather database  
        if (table_name == 'hourly_weather'):
            df = table_to_df_dict[table_name]
            insert_query2 = """
            INSERT INTO hourly_weather(DATE, hour,
                                       HourlyPrecipitation,
                                       HourlyWindSpeed)
            VALUES (?, ?, ?, ?)
            """

            # Loop through DataFrame rows and insert each row into the table
            data = []
            for index, row in df.iterrows():
                DATE = row['DATE'].strftime('%Y-%m-%d') 
                data.append((
                DATE,
                row['hour'],
                row['HourlyPrecipitation'], row['HourlyWindSpeed']
                ))
        
            with connection:
                connection.executemany(insert_query2,data)

        #add data to daily weather database       
        if (table_name == 'daily_weather'):
            df = table_to_df_dict[table_name]
            insert_query3 = """
            INSERT INTO daily_weather(DATE,
                                      DailyPrecipitation,
                                      DailyAverageWindSpeed,
                                      DailySnowfall)
            VALUES (?, ?, ?, ?)
            """
    
            #loop through DataFrame rows and insert each row into the table
            data = []
            for index, row in df.iterrows():
                DATE = row['DATE'].strftime('%Y-%m-%d') 
                data.append((
                DATE,
                row['DailyPrecipitation'],
                row['DailyAverageWindSpeed'], row['DailySnowfall']
                ))
        
            with connection:
                connection.executemany(insert_query3,data)

        #add data to Uber database 
        if (table_name == 'uber_trips'):
            df = table_to_df_dict[table_name]
            insert_query4 = """
            INSERT INTO uber_trips (request_datetime,
                                    pickup_datetime,
                                    dropoff_datetime,
                                    trip_miles,
                                    trip_time_seconds,
                                    pickup_longitude,
                                    pickup_latitude,
                                    dropoff_longitude,
                                    dropoff_latitude,
                                    base_passenger_fare,
                                    tolls,
                                    black_car_fund,
                                    sales_tax,
                                    congestion_surcharge,
                                    airport_fee,
                                    tips,
                                    all_surcharge)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """
    
            #loop through DataFrame rows and insert each row into the table
            data = []
            for index, row in df.iterrows():
                request_datetime = row['request_datetime'].strftime('%Y-%m-%d %H:%M:%S') 
                pickup_datetime = row['pickup_datetime'].strftime('%Y-%m-%d %H:%M:%S') 
                dropoff_datetime = row['dropoff_datetime'].strftime('%Y-%m-%d %H:%M:%S') 
                data.append((request_datetime,
                pickup_datetime, dropoff_datetime,
                row['trip_miles'],row['trip_time_seconds'], row['pickup_longitude'],
                row['pickup_latitude'], row['dropoff_longitude'],row['dropoff_latitude'], 
                row['base_passenger_fare'], row['tolls'], row['black_car_fund'],
                row['sales_tax'], row['congestion_surcharge'], row['airport_fee'], 
                row['tips'], row['all_surcharge']
                ))
        
            with connection:
                connection.executemany(insert_query4,data)

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query: str, outfile: str)-> None:
    """
    Write the SQL query to a specified file.

    Parameters:
        query (str): The SQL query to be written to the file.
        outfile (str): The path to the output file where the query should be stored.

    Returns:
        None
    """
    with open(outfile, 'w') as f:
        f.write(query)

### Query 1

In [ ]:
QUERY_1_FILENAME = "the_most_popular_hour_to_take_a_taxi.sql"

QUERY_1 = """
SELECT 
strftime('%H', pickup_datetime) AS hour_of_day,
COUNT(pickup_datetime) AS number_of_taxi
FROM taxi_trips
WHERE pickup_datetime >= '2020-01-01' AND pickup_datetime < '2024-09-01'
GROUP BY hour_of_day
ORDER BY number_of_taxi DESC
"""

In [ ]:
#show the result
pd.read_sql(QUERY_1, con=connection)

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

### Query 2

In [ ]:
QUERY_2_FILENAME = "the_most_popular_day_of_the_week_to_take_an_Uber.sql"

QUERY_2 = """
SELECT 
strftime('%w', pickup_datetime) AS day_of_week,
COUNT(pickup_datetime) AS number_of_uber
FROM uber_trips
WHERE pickup_datetime >= '2020-01-01' AND pickup_datetime < '2024-09-01'
GROUP BY day_of_week
ORDER BY number_of_uber DESC
"""

In [ ]:
#show the result
pd.read_sql(QUERY_2, con=connection)

In [ ]:
write_query_to_file(QUERY_2, QUERY_2_FILENAME)

### Query 3

In [ ]:
QUERY_3_FILENAME = "the_95_percentile_of_trip_distance_in_January_2024.sql"

QUERY_3 = """
SELECT trip_distance
FROM (
    SELECT trip_distance,
    ROW_NUMBER() OVER (ORDER BY trip_distance ASC) AS row_num,
    COUNT(*) OVER () AS total_count
FROM (
    SELECT trip_distance
    FROM taxi_trips
    WHERE pickup_datetime >= '2024-01-01' AND pickup_datetime < '2024-02-01'
    UNION ALL
    SELECT trip_miles AS trip_distance
    FROM uber_trips
    WHERE pickup_datetime >= '2024-01-01' AND pickup_datetime < '2024-02-01') 
) 
WHERE row_num = FLOOR(total_count * 0.95)
"""

In [ ]:
#show the result
pd.read_sql(QUERY_3, con=connection)

In [ ]:
write_query_to_file(QUERY_3, QUERY_3_FILENAME)

### Query 4

In [ ]:
QUERY_4_FILENAME = "the_weather_like_for_the_busiest_days_in_2023.sql"

QUERY_4 = """
SELECT ride_date, number_of_rides, average_trip_distance,
DailyPrecipitation/24 AS Average_DailyPrecipitation ,DailyAverageWindSpeed
FROM (
    SELECT ride_date, 
    COUNT(*) AS number_of_rides,
    AVG(trip_distance) AS average_trip_distance
    FROM(
        SELECT strftime('%Y-%m-%d', pickup_datetime) AS ride_date, trip_distance
        FROM taxi_trips
        WHERE pickup_datetime >= '2023-01-01' AND pickup_datetime < '2024-01-01'
        UNION ALL
        SELECT strftime('%Y-%m-%d', pickup_datetime) AS ride_date, trip_miles AS trip_distance
        FROM uber_trips
        WHERE pickup_datetime >= '2023-01-01' AND pickup_datetime < '2024-01-01')
    GROUP BY ride_date
    ORDER BY number_of_rides DESC
    LIMIT 10) AS total_rides
JOIN daily_weather AS w ON total_rides.ride_date = w.DATE
"""

In [ ]:
#show the result
pd.read_sql(QUERY_4, con=connection)

In [ ]:
write_query_to_file(QUERY_4, QUERY_4_FILENAME)

### Query 5

In [ ]:
QUERY_5_FILENAME = "number_of_rides_were_hired_during_snow_days.sql"

QUERY_5 = """
SELECT ride_date, DailySnowfall, number_of_rides
FROM (daily_weather as w
JOIN(
SELECT ride_date,
COUNT(*) AS number_of_rides
FROM(
SELECT strftime('%Y-%m-%d', pickup_datetime) AS ride_date
FROM taxi_trips
WHERE pickup_datetime >= '2020-01-01' AND pickup_datetime < '2024-09-01'
UNION ALL
SELECT strftime('%Y-%m-%d', pickup_datetime) AS ride_date
FROM uber_trips
WHERE pickup_datetime >= '2020-01-01' AND pickup_datetime < '2024-09-01')
GROUP BY ride_date) AS t on w.DATE = t.ride_date)
ORDER BY DailySnowfall DESC
LIMIT 10
"""

In [ ]:
#show the result
pd.read_sql(QUERY_5, con=connection)

In [ ]:
write_query_to_file(QUERY_5, QUERY_5_FILENAME)

### Query 6

In [ ]:
QUERY_6 = """
SELECT w.date,w.hour,
    COALESCE(total_rides, 0) AS total_rides,
    COALESCE(total_precipitation, 0) AS total_precipitation,
    COALESCE(avg_wind_speed, 0) AS avg_wind_speed
FROM (
SELECT strftime('%Y-%m-%d', DATE) AS date,    
    hour,  
    SUM(HourlyPrecipitation) AS total_precipitation,
    AVG(HourlyWindSpeed) AS avg_wind_speed
FROM hourly_weather
WHERE DATE >= '2023-09-25' AND DATE <= '2023-10-03'
GROUP BY date, hour) AS w
LEFT JOIN(
    SELECT date, 
    hour, 
    SUM(number_of_taxi + number_of_uber) AS total_rides
    FROM (
    SELECT strftime('%Y-%m-%d', pickup_datetime) AS date,
        strftime('%H', pickup_datetime) AS hour,
        COUNT(*) AS number_of_taxi,  
        0 AS number_of_uber     
    FROM taxi_trips
    WHERE pickup_datetime >= '2023-09-25' AND pickup_datetime <= '2023-10-03'
    GROUP BY date, hour
        
    UNION ALL
        
    SELECT strftime('%Y-%m-%d', pickup_datetime) AS date,
        strftime('%H', pickup_datetime) AS hour,
        COUNT(*) AS number_of_uber,    
        0 AS number_of_taxi
    FROM uber_trips
    WHERE pickup_datetime >= '2023-09-25' AND pickup_datetime <= '2023-10-03'
    GROUP BY date, hour)
    GROUP BY date, hour) AS t
ON w.date = t.date AND w.hour = t.hour
ORDER BY w.date, w.hour
"""

In [ ]:
#show the result
pd.read_sql(QUERY_6, con=connection)

In [ ]:
write_query_to_file(QUERY_6, QUERY_6_FILENAME)

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
def plot_most_popular_hour_to_take_a_taxi(dataframe: pd.DataFrame) -> None:
    figure, axes = plt.subplots(figsize=(20, 10))
    hours = dataframe['hour_of_day']
    taxi_count = dataframe['number_of_taxi']

    axes.bar(hours, taxi_count)
    axes.set_xlabel('Hour of Day in 24-hour Format')
    axes.set_xticks(range(len(hours)))
    axes.set_ylabel('Number of Taxi')
    axes.set_title("Most Popular Hour to Take a Taxi")
    
    plt.show()

In [ ]:
def get_data_for_most_popular_hour_to_take_a_taxi() -> pd.DataFrame:
    dataframe = pd.read_sql(QUERY_1, con=connection)
    return dataframe

In [ ]:
some_dataframe = get_data_for_most_popular_hour_to_take_a_taxi()
plot_most_popular_hour_to_take_a_taxi(some_dataframe)

### Visualization 2

In [ ]:
QUERY_AVERAGE_DISTANCE_TRAVELED_PER_MONTH_FILENAME = "average_distance_traveled_per_month.sql"

QUERY_AVERAGE_DISTANCE_TRAVELED_PER_MONTH = """
SELECT 
    strftime('%m', pickup_datetime) AS month,
    trip_distance AS distance
FROM taxi_trips
WHERE pickup_datetime >= '2020-01-01' AND pickup_datetime < '2024-09-01'
UNION ALL
SELECT
    strftime('%m', pickup_datetime) AS month,
    trip_miles AS distance
FROM uber_trips
WHERE pickup_datetime >= '2020-01-01' AND pickup_datetime < '2024-09-01'
"""

In [ ]:
def plot_average_distance_traveled_per_month(dataframe: pd.DataFrame) -> None:

    grouped = dataframe.groupby('month')['distance'].mean().reset_index()
    grouped['month'] = grouped['month'].astype(int)
    grouped = grouped.sort_values('month')
    
    x = grouped['month']  
    y = grouped['distance'] 
    dy = 0.9
    
    plt.errorbar(x, y, yerr=dy, fmt='.-', label='Average Distance')
    plt.xticks(range(1, 13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
    plt.title('Average Distance Traveled Per Month')
    plt.xlabel('Month')
    plt.ylabel('Average Distance')
    plt.legend()
    plt.show()

In [ ]:
def get_data_for_average_distance_traveled_per_month() -> pd.DataFrame:
    dataframe = pd.read_sql(QUERY_AVERAGE_DISTANCE_TRAVELED_PER_MONTH, con=connection)
    return dataframe

In [ ]:
some_dataframe = get_data_for_average_distance_traveled_per_month()
plot_average_distance_traveled_per_month(some_dataframe)

### Visualization 3

In [ ]:
QUERY_LGA_FILENAME = "the_most_popular_day_of_the_week_for_drop_offs_in_LGA.sql"

QUERY_LGA = f"""
SELECT 
    strftime('%w', pickup_datetime) AS day_of_week,
    COUNT(pickup_datetime) AS number_of_trips
FROM uber_trips
WHERE pickup_datetime >= '2020-01-01' 
    AND pickup_datetime < '2024-09-01'
    AND dropoff_latitude BETWEEN {LGA_BOX_COORDS[0][0]} AND {LGA_BOX_COORDS[1][0]}
    AND dropoff_longitude BETWEEN {LGA_BOX_COORDS[0][1]} AND {LGA_BOX_COORDS[1][1]}
GROUP BY day_of_week
UNION ALL
SELECT 
    strftime('%w', pickup_datetime) AS day_of_week,
    COUNT(pickup_datetime) AS number_of_trips
FROM taxi_trips
WHERE pickup_datetime >= '2020-01-01' 
    AND pickup_datetime < '2024-09-01'
    AND dropoff_latitude BETWEEN {LGA_BOX_COORDS[0][0]} AND {LGA_BOX_COORDS[1][0]}
    AND dropoff_longitude BETWEEN {LGA_BOX_COORDS[0][1]} AND {LGA_BOX_COORDS[1][1]}
GROUP BY day_of_week
"""

In [ ]:
QUERY_LGA_FILENAME = "the_most_popular_day_of_the_week_for_drop_offs_in_JFK.sql"
QUERY_JFK = f"""
SELECT 
    strftime('%w', pickup_datetime) AS day_of_week,
    COUNT(pickup_datetime) AS number_of_trips
FROM uber_trips
WHERE pickup_datetime >= '2020-01-01' 
    AND pickup_datetime < '2024-09-01'
    AND dropoff_latitude BETWEEN {JFK_BOX_COORDS[0][0]} AND {JFK_BOX_COORDS[1][0]}
    AND dropoff_longitude BETWEEN {JFK_BOX_COORDS[0][1]} AND {JFK_BOX_COORDS[1][1]}
GROUP BY day_of_week
UNION ALL
SELECT 
    strftime('%w', pickup_datetime) AS day_of_week,
    COUNT(pickup_datetime) AS number_of_trips
FROM taxi_trips
WHERE pickup_datetime >= '2020-01-01' 
    AND pickup_datetime < '2024-09-01'
    AND dropoff_latitude BETWEEN {JFK_BOX_COORDS[0][0]} AND {JFK_BOX_COORDS[1][0]}
    AND dropoff_longitude BETWEEN {JFK_BOX_COORDS[0][1]} AND {JFK_BOX_COORDS[1][1]}
GROUP BY day_of_week
"""

In [ ]:
QUERY_LGA_FILENAME = "the_most_popular_day_of_the_week_for_drop_offs_in_EWR.sql"

QUERY_EWR = f"""
SELECT 
    strftime('%w', pickup_datetime) AS day_of_week,
    COUNT(pickup_datetime) AS number_of_trips
FROM uber_trips
WHERE pickup_datetime >= '2020-01-01' 
    AND pickup_datetime < '2024-09-01'
    AND dropoff_latitude BETWEEN {EWR_BOX_COORDS[0][0]} AND {EWR_BOX_COORDS[1][0]}
    AND dropoff_longitude BETWEEN {EWR_BOX_COORDS[0][1]} AND {EWR_BOX_COORDS[1][1]}
GROUP BY day_of_week
UNION ALL
SELECT 
    strftime('%w', pickup_datetime) AS day_of_week,
    COUNT(pickup_datetime) AS number_of_trips
FROM taxi_trips
WHERE pickup_datetime >= '2020-01-01' 
    AND pickup_datetime < '2024-09-01'
    AND dropoff_latitude BETWEEN {EWR_BOX_COORDS[0][0]} AND {EWR_BOX_COORDS[1][0]}
    AND dropoff_longitude BETWEEN {EWR_BOX_COORDS[0][1]} AND {EWR_BOX_COORDS[1][1]}
GROUP BY day_of_week
"""

In [ ]:
def plot_popular_day_for_drop_offs_in_airports(dataframe: pd.DataFrame) -> None:
    #map day_of_week integers to day names
    day_names = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
    dataframe['day_name'] = dataframe['day_of_week'].astype(int).apply(lambda x: day_names[x])

    #group by airport and day name
    grouped_data = dataframe.groupby(['airport', 'day_name'])['number_of_trips'].sum().reset_index()

    #pivot data for easier plotting
    pivot_data = grouped_data.pivot(index='day_name', columns='airport', values='number_of_trips').fillna(0)
    pivot_data = pivot_data.reindex(day_names)  # Ensure days are in correct order

    #plot a bar graph for each airport
    pivot_data.plot(kind='bar', figsize=(12, 8), width=0.8)

    #add titles and labels
    plt.title('Most Popular Day of Week for Drop-Offs in Airports (LGA, JFK, EWR)', fontsize=16)
    plt.xlabel('Day of the Week', fontsize=14)
    plt.ylabel('Number of Trips', fontsize=14)
    plt.xticks(rotation=45, fontsize=12)
    plt.legend(title='Airport', fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    #show plot
    plt.tight_layout()
    plt.show()

In [ ]:
def get_data_for_popular_day_for_drop_offs_in_airports() -> pd.DataFrame:
    lga_data = pd.read_sql(QUERY_LGA, connection)
    lga_data['airport'] = 'LGA'
    
    jfk_data = pd.read_sql(QUERY_JFK, connection)
    jfk_data['airport'] = 'JFK'
    
    ewr_data = pd.read_sql(QUERY_EWR, connection)
    ewr_data['airport'] = 'EWR'
    
    dataframe = pd.concat([lga_data, jfk_data, ewr_data])

    return dataframe

In [ ]:
some_dataframe = get_data_for_popular_day_for_drop_offs_in_airports()
plot_popular_day_for_drop_offs_in_airports(some_dataframe)

### Visualization 4

In [ ]:
QUERY_MONTHLY_TOTAL_FARE_FILENAME = 'monthly_earned_total_fares_for_uber_and_taxi.sql'
QUERY_MONTHLY_TOTAL_FARE_UBER = """
SELECT
    base_passenger_fare AS base_fare,
    all_surcharge AS all_surcharge,
    tolls AS tolls,
    sales_tax AS tax,
    base_passenger_fare + all_surcharge + tolls + sales_tax AS total_fare,
    strftime('%Y-%m', pickup_datetime) AS month_year
FROM uber_trips
WHERE pickup_datetime >= '2020-01-01' 
    AND pickup_datetime < '2024-09-01'
"""
QUERY_MONTHLY_TOTAL_FARE_TAXI = """
SELECT
    fare_amount AS base_fare, 
    surcharge AS all_surcharge, 
    tolls_amount AS tolls, 
    taxes AS tax,
    fare_amount + surcharge + tolls_amount + taxes AS total_fare,
    strftime('%Y-%m', pickup_datetime) AS month_year
FROM taxi_trips
WHERE pickup_datetime >= '2020-01-01' 
    AND pickup_datetime < '2024-09-01'
"""

In [ ]:
def plot_monthly_earned_total_fares_for_uber_and_taxi(uber_fare_data: pd.DataFrame, taxi_fare_data: pd.DataFrame) -> None:
    #x-axis positions
    all_months = pd.date_range(start='2020-01', end='2024-08', freq='MS')
    months = all_months.strftime('%Y-%m')
    x = np.arange(len(months))  

    #sorted by month_year
    uber_fare_data = uber_fare_data.groupby('month_year', as_index=False).sum()
    taxi_fare_data = taxi_fare_data.groupby('month_year', as_index=False).sum()


    fig, axs = plt.subplots(2, 1, figsize=(16, 12))

    #Uber stacked bar chart
    axs[0].bar(x, uber_fare_data['base_fare'], label='Uber Base Fare', color='lightblue')
    axs[0].bar(x, uber_fare_data['all_surcharge'], bottom=uber_fare_data['base_fare'], label='Uber Surcharge', color='orange')
    axs[0].bar(x, uber_fare_data['tolls'], bottom=uber_fare_data['base_fare'] + uber_fare_data['all_surcharge'], label='Uber Tolls', color='green')
    axs[0].bar(x, uber_fare_data['tax'], bottom=uber_fare_data['base_fare'] + uber_fare_data['all_surcharge'] + uber_fare_data['tolls'], label='Uber Tax', color='salmon')
    axs[0].set_title('Monthly Earned Total Fares for Uber (2020-2024)')
    axs[0].set_ylabel('Total Amount ($)')
    axs[0].legend(loc='upper left')
    axs[0].grid(axis='y', linestyle='--', alpha=0.7)

    #Taxi stacked bar chart
    axs[1].bar(x, taxi_fare_data['base_fare'], label='Taxi Base Fare', color='lightblue')
    axs[1].bar(x, taxi_fare_data['all_surcharge'], bottom=taxi_fare_data['base_fare'], label='Taxi Surcharge', color='orange')
    axs[1].bar(x, taxi_fare_data['tolls'], bottom=taxi_fare_data['base_fare'] + taxi_fare_data['all_surcharge'], label='Taxi Tolls', color='green')
    axs[1].bar(x, taxi_fare_data['tax'], bottom=taxi_fare_data['base_fare'] + taxi_fare_data['all_surcharge'] + taxi_fare_data['tolls'], label='Taxi Tax', color='salmon')
    axs[1].set_title('Monthly Earned Total Fares for Yellow Taxis (2020-2024)')
    axs[1].set_xlabel('Month')
    axs[1].set_ylabel('Total Amount ($)')
    axs[1].legend(loc='upper left')
    axs[1].grid(axis='y', linestyle='--', alpha=0.7)

    #x-axis labels for both charts
    axs[0].set_xticks(x)
    axs[0].set_xticklabels(months, rotation=45)
    axs[1].set_xticks(x)
    axs[1].set_xticklabels(months, rotation=45)

    plt.tight_layout()
    plt.show()

In [ ]:
def get_data_for_monthly_earned_total_fares_for_uber_and_taxi() -> Tuple[pd.DataFrame, pd.DataFrame]:
    uber_fare_data = pd.read_sql(QUERY_MONTHLY_TOTAL_FARE_UBER, con=connection)
    taxi_fare_data = pd.read_sql(QUERY_MONTHLY_TOTAL_FARE_TAXI, con=connection)
    
    return uber_fare_data, taxi_fare_data

In [ ]:
uber_fare_data, taxi_fare_data = get_data_for_monthly_earned_total_fares_for_uber_and_taxi()
plot_monthly_earned_total_fares_for_uber_and_taxi(uber_fare_data, taxi_fare_data)

### Visualization 5

In [ ]:
QUERY_TAXI_TIP_FILENAME = 'amount_of_taxi_tip_affcted_by_distance_and_precipitation.sql'
QUERY_TAXI_TIP = """
SELECT 
    trip_distance AS distance,
    tip_amount AS tip,
    pickup_datetime,
    daily_weather.DailyPrecipitation AS precipitation
FROM taxi_trips
JOIN daily_weather
    ON DATE(taxi_trips.pickup_datetime) = daily_weather.DATE
WHERE pickup_datetime >= '2022-01-01' 
    AND pickup_datetime < '2024-01-01'
"""

QUERY_UBER_TIP_FILENAME = 'amount_of_uber_tip_affcted_by_distance_and_precipitation.sql'
QUERY_UBER_TIP = """
SELECT 
    trip_miles AS distance,
    tips AS tip,
    pickup_datetime,
    daily_weather.DailyPrecipitation AS precipitation
FROM uber_trips
JOIN daily_weather
    ON DATE(uber_trips.pickup_datetime) = daily_weather.DATE
WHERE pickup_datetime >= '2022-01-01' 
    AND pickup_datetime < '2024-01-01'
"""

In [ ]:
def plot_amount_of_uber_tip_affcted_by_distance_and_precipitation(uber_tip_data: pd.DataFrame, taxi_tip_data: pd.DataFrame) -> None:
    fig, axs = plt.subplots(2, 2)
    #distance
    axs[0, 0].scatter(taxi_data['distance'], taxi_data['tip'])
    axs[0, 0].set_title('Yellow Taxi: Tip vs. Distance')
    axs[0, 0].set_xlabel('Distance in miles')
    axs[0, 0].set_ylabel('Tip')

    axs[0, 1].scatter(uber_data['distance'], uber_data['tip'])
    axs[0, 1].set_title('Uber: Tip vs. Distance')
    axs[0, 1].set_xlabel('Distance in miles')
    axs[0, 1].set_ylabel('Tip')

    #precipitation
    axs[1, 0].scatter(taxi_data['precipitation'], taxi_data['tip'])
    axs[1, 0].set_title('Yellow Taxi: Tip vs. Precipitation')
    axs[1, 0].set_xlabel('Precipitation')
    axs[1, 0].set_ylabel('Tip')

    axs[1, 1].scatter(uber_data['precipitation'], uber_data['tip'])
    axs[1, 1].set_title('Uber: Tip vs. Precipitation')
    axs[1, 1].set_xlabel('Precipitation')
    axs[1, 1].set_ylabel('Tip')
    plt.tight_layout()
    plt.show

In [ ]:
def get_data_for_amount_of_uber_tip_affcted_by_distance_and_precipitation5() -> Tuple[pd.DataFrame, pd.DataFrame]:
    uber_tip_data = pd.read_sql(QUERY_UBER_TIP, con=connection)
    taxi_tip_data = pd.read_sql(QUERY_TAXI_TIP, con=connection)
    return uber_tip_data, taxi_tip_data

In [ ]:
uber_tip_data, taxi_tip_data = get_data_for_amount_of_uber_tip_affcted_by_distance_and_precipitation5()
plot_amount_of_uber_tip_affcted_by_distance_and_precipitation(uber_tip_data, taxi_tip_data)

### Visualization 6

In [ ]:
QUERY_LOCATION_FILENAME = 'all_hired_trips_in_2020_over_map'
QUERY_LOCATION = """
SELECT 
    pickup_latitude,
    pickup_longitude,
    dropoff_latitude,
    dropoff_longitude
FROM taxi_trips
WHERE pickup_datetime >= '2020-01-01' AND pickup_datetime < '2021-01-01'
UNION ALL
SELECT 
    pickup_latitude,
    pickup_longitude,
    dropoff_latitude,
    dropoff_longitude
FROM uber_trips
WHERE pickup_datetime >= '2020-01-01' AND pickup_datetime < '2021-01-01';
"""

In [ ]:
def plot_all_hired_trips_in_2020_over_map(dataframe: pd.DataFrame) -> None:
    #in the NYC area
    dataframe = dataframe[
        (dataframe['pickup_latitude'] >= NEW_YORK_BOX_COORDS[0][0]) &
        (dataframe['dropoff_latitude'] >= NEW_YORK_BOX_COORDS[0][0]) &
        (dataframe['pickup_latitude'] <= NEW_YORK_BOX_COORDS[1][0]) &
        (dataframe['dropoff_latitude'] <= NEW_YORK_BOX_COORDS[1][0]) &
        (dataframe['pickup_longitude'] >= NEW_YORK_BOX_COORDS[0][1]) &
        (dataframe['dropoff_longitude'] >= NEW_YORK_BOX_COORDS[0][1]) &
        (dataframe['pickup_longitude'] <= NEW_YORK_BOX_COORDS[1][1]) &
        (dataframe['dropoff_longitude'] <= NEW_YORK_BOX_COORDS[1][1]) 
    ]
    center_latitude = (NEW_YORK_BOX_COORDS[0][0] + NEW_YORK_BOX_COORDS[1][0]) / 2
    center_longitude = (NEW_YORK_BOX_COORDS[0][1] + NEW_YORK_BOX_COORDS[1][1]) / 2
    nyc_map = folium.Map(location=[center_latitude, center_longitude])
    
    pickup_locations = dataframe[['pickup_latitude', 'pickup_longitude']].values.tolist()
    dropoff_locations = dataframe[['dropoff_latitude', 'dropoff_longitude']].values.tolist()
    all_location = pickup_locations + dropoff_locations

    HeatMap(all_location, radius = 10).add_to(nyc_map)
    return nyc_map

In [ ]:
def get_data_for_all_hired_trips_in_2020_over_map() -> pd.DataFrame:
    dataframe = pd.read_sql(QUERY_LOCATION, con=connection)
    return dataframe

In [ ]:
some_dataframe = get_data_for_all_hired_trips_in_2020_over_map()
plot_all_hired_trips_in_2020_over_map(some_dataframe)

Annotation: Folium utilized both drop-off and pick-up locations.